In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import os

In [2]:
shops = pd.read_csv("data/shops.csv")
item_categories = pd.read_csv("data/item_categories.csv")
sales_train = pd.read_csv("data/sales_train.csv")
items = pd.read_csv("data/items.csv")

test = pd.read_csv("data/test.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")

Stack the train and test data together

In [3]:
test['date_block_num'] = 34
train_test = sales_train.append(test.drop(columns=['ID']))
train_test.tail(2)

C:\Users\ChenP\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


,date,date_block_num,item_cnt_day,item_id,item_price,shop_id
214198,NaN,34,NaN,19648,NaN,45
214199,NaN,34,NaN,969,NaN,45


### a. Aggregate Basic Features

Aggregate sales to monthly level and clip the value, and also join with other features

In [4]:
# Aggregate sales to monthly level
train_master = train_test.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_day': ['sum']})
train_master.columns = ['item_cnt_sum']
train_master.reset_index(inplace=True)
train_master['item_cnt_sum'] = np.clip(train_master['item_cnt_sum'], 0, 20)

# Attach the item_category_id, shop_id, shop_name columns
train_master = train_master.merge(items[['item_id','item_category_id']], on='item_id', how='left')
train_master = train_master.merge(shops, on='shop_id', how='left')
train_master = train_master.merge(item_categories[['item_category_id']], on='item_category_id', how='left')

train_master.head(2)

,date_block_num,shop_id,item_id,item_cnt_sum,item_category_id,shop_name
0,0,0,32,6.0,40,"!Якутск Орджоникидзе, 56 фран"
1,0,0,33,3.0,37,"!Якутск Орджоникидзе, 56 фран"


Notice that we do not have all the (date_block_num, shop_id, item_id) pairs, if the sales = 0. Therefore, we need to fill in the missing pairs.

In [5]:
from itertools import product

months = train_master[train_master.date_block_num <= 33]['date_block_num'].unique()

cartesian = []
for month in months:
    shops_in_month = train_master.loc[train_master['date_block_num']==month, 'shop_id'].unique()
    items_in_month = train_master.loc[train_master['date_block_num']==month, 'item_id'].unique()
    cartesian.append(np.array(list(product(*[shops_in_month, items_in_month, [month]])), dtype='int32'))
    
cartesian_df = pd.DataFrame(np.vstack(cartesian), columns = ['shop_id', 'item_id', 'date_block_num'], dtype=np.int32)
print(cartesian_df.shape)

(10913850, 3)


The generated cartesian_df dataframe will contain all the unique (shop_id, item_id, date_block_num) pairs

Append the test set into the cartesian_df

In [6]:
test['date_block_num'] = 34
cartesian_df = cartesian_df.append(test.drop(columns=['ID']))

We merge it back to the train dataframe and fill NaN with zeros to include zeros in the sales record

In [7]:
new_train = cartesian_df.merge(train_master[['date_block_num','shop_id', 'item_id','item_cnt_sum']], 
                               on=['date_block_num','shop_id','item_id'], how='left').fillna(0)
new_train.head()

,shop_id,item_id,date_block_num,item_cnt_sum
0,0,32,0,6.0
1,0,33,0,3.0
2,0,35,0,1.0
3,0,43,0,1.0
4,0,51,0,2.0


Append feature columns and deduce the city feature from the shop name

In [8]:
# Attach the item_category_id, shop_id, shop_name columns
new_train = new_train.merge(items[['item_id','item_category_id']], on='item_id', how='left')
new_train = new_train.merge(shops, on='shop_id', how='left')
new_train = new_train.merge(item_categories[['item_category_id']], on='item_category_id', how='left')

# find the city attribute from the first word of the shop name
new_train['city'] = new_train['shop_name'].str.split(' ').map(lambda x : x[0])
new_train.drop(columns=['shop_name'], inplace=True)
new_train.reset_index(drop=True, inplace=True)

new_train.head(2)

,shop_id,item_id,date_block_num,item_cnt_sum,item_category_id,city
0,0,32,0,6.0,40,!Якутск
1,0,33,0,3.0,37,!Якутск


### b. Add features: item_id related features
- item_id
- shop_id
- last 1, 2, 3, 6, 12 month's sale for the same item_id in the same shop_id
- last 1, 2, 3, 6, 12 month's mean sale for the same item_id in all shops

First create a function to add lag feature more easily

In [9]:
def lag_feature(df, df_shift, lags, on, col):
    tmp = df_shift[on + [col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = on + [col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = df.merge(shifted, on=on, how='left')
    return df

Then, lag the target variable

In [10]:
new_train = lag_feature(new_train, new_train, lags=[1,2,3,6,12], on=['date_block_num','shop_id','item_id'], col='item_cnt_sum')
new_train.head(2)

,shop_id,item_id,date_block_num,item_cnt_sum,item_category_id,city,item_cnt_sum_lag_1,item_cnt_sum_lag_2,item_cnt_sum_lag_3,item_cnt_sum_lag_6,item_cnt_sum_lag_12
0,0,32,0,6.0,40,!Якутск,NaN,NaN,NaN,NaN,NaN
1,0,33,0,3.0,37,!Якутск,NaN,NaN,NaN,NaN,NaN


To lag the mean item_cnt_sum for the same item_id over all shops, we first compute the average and then lag

In [11]:
df = train_master.groupby(['date_block_num', 'item_id']).agg({'item_cnt_sum':['mean']})
df.columns = ['item_cnt_sum_avg_by_shop']
df.reset_index(inplace=True)

new_train = lag_feature(new_train, df, lags=[1,2,3,6,12], on=['date_block_num','item_id'], col='item_cnt_sum_avg_by_shop')
new_train.head(2)

,shop_id,item_id,date_block_num,item_cnt_sum,item_category_id,city,item_cnt_sum_lag_1,item_cnt_sum_lag_2,item_cnt_sum_lag_3,item_cnt_sum_lag_6,item_cnt_sum_lag_12,item_cnt_sum_avg_by_shop_lag_1,item_cnt_sum_avg_by_shop_lag_2,item_cnt_sum_avg_by_shop_lag_3,item_cnt_sum_avg_by_shop_lag_6,item_cnt_sum_avg_by_shop_lag_12
0,0,32,0,6.0,40,!Якутск,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,33,0,3.0,37,!Якутск,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### c. Add features: Categorical related features
- item_category_id
- last 1, 2, 3, 6, 12 months' sale for items with the same item_category_id in the same shop
- last 1, 2, 3, 6, 12 months' sale for items with the same item_category_id averaged among all shops

In [12]:
df = train_master.groupby(['date_block_num', 'shop_id', 'item_category_id']).agg({'item_cnt_sum':['mean']})
df.columns = ['item_cnt_sum_avg_by_cat']
df.reset_index(inplace=True)

new_train = lag_feature(new_train, df, lags=[1,2,3,6,12], 
                        on=['date_block_num','shop_id','item_category_id'], col='item_cnt_sum_avg_by_cat')
new_train.head(2)

,shop_id,item_id,date_block_num,item_cnt_sum,item_category_id,city,item_cnt_sum_lag_1,item_cnt_sum_lag_2,item_cnt_sum_lag_3,item_cnt_sum_lag_6,...,item_cnt_sum_avg_by_shop_lag_1,item_cnt_sum_avg_by_shop_lag_2,item_cnt_sum_avg_by_shop_lag_3,item_cnt_sum_avg_by_shop_lag_6,item_cnt_sum_avg_by_shop_lag_12,item_cnt_sum_avg_by_cat_lag_1,item_cnt_sum_avg_by_cat_lag_2,item_cnt_sum_avg_by_cat_lag_3,item_cnt_sum_avg_by_cat_lag_6,item_cnt_sum_avg_by_cat_lag_12
0,0,32,0,6.0,40,!Якутск,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,33,0,3.0,37,!Якутск,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df = train_master.groupby(['date_block_num', 'item_category_id']).agg({'item_cnt_sum':['mean']})
df.columns = ['item_cnt_sum_avg_by_cat_by_shop']
df.reset_index(inplace=True)

new_train = lag_feature(new_train, df, lags=[1,2,3,6,12], 
                        on=['date_block_num','item_category_id'], col='item_cnt_sum_avg_by_cat_by_shop')
new_train.head(2)

,shop_id,item_id,date_block_num,item_cnt_sum,item_category_id,city,item_cnt_sum_lag_1,item_cnt_sum_lag_2,item_cnt_sum_lag_3,item_cnt_sum_lag_6,...,item_cnt_sum_avg_by_cat_lag_1,item_cnt_sum_avg_by_cat_lag_2,item_cnt_sum_avg_by_cat_lag_3,item_cnt_sum_avg_by_cat_lag_6,item_cnt_sum_avg_by_cat_lag_12,item_cnt_sum_avg_by_cat_by_shop_lag_1,item_cnt_sum_avg_by_cat_by_shop_lag_2,item_cnt_sum_avg_by_cat_by_shop_lag_3,item_cnt_sum_avg_by_cat_by_shop_lag_6,item_cnt_sum_avg_by_cat_by_shop_lag_12
0,0,32,0,6.0,40,!Якутск,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,33,0,3.0,37,!Якутск,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### d. Add features: month and city label

Add a month feature

In [14]:
new_train['month'] = new_train['date_block_num'] % 12

Label encode the categorical variables

In [15]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
new_train['city_label'] = le.fit_transform(new_train['city'])
new_train.drop(['city'], axis = 1, inplace=True)
new_train.head(2)

,shop_id,item_id,date_block_num,item_cnt_sum,item_category_id,item_cnt_sum_lag_1,item_cnt_sum_lag_2,item_cnt_sum_lag_3,item_cnt_sum_lag_6,item_cnt_sum_lag_12,...,item_cnt_sum_avg_by_cat_lag_3,item_cnt_sum_avg_by_cat_lag_6,item_cnt_sum_avg_by_cat_lag_12,item_cnt_sum_avg_by_cat_by_shop_lag_1,item_cnt_sum_avg_by_cat_by_shop_lag_2,item_cnt_sum_avg_by_cat_by_shop_lag_3,item_cnt_sum_avg_by_cat_by_shop_lag_6,item_cnt_sum_avg_by_cat_by_shop_lag_12,month,city_label
0,0,32,0,6.0,40,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,0,33,0,3.0,37,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


### e. Save the dataset

In [17]:
new_train.to_pickle('train_test.pkl')

del cartesian_df
del items
del shops
del sales_train
del train_master
del train_test

In [18]:
import gc

gc.collect()

61